In [14]:
import pandas as pd
import os
import torch
import itertools
from scipy import io

In [3]:
torch.cuda.is_available()
torch.__version__

'1.9.0'

In [10]:
# get a list of all the split-seq outputs:
splitseq_dir = "/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/"

# batch directories:
batch_dirs = ["{}{}/".format(splitseq_dir, d) for d in os.listdir(splitseq_dir)]

# sublibrary directories:
sublib_dirs = [["{}{}/all-well/DGE_unfiltered/".format(b, s)  for s in os.listdir(b)] for b in batch_dirs]
sublib_dirs = list(itertools.chain.from_iterable(sublib_dirs))

In [15]:
# loop through each sublibrary:
for cur_dir in sublib_dirs:
    
    print(cur_dir)
    
    # read the cell-meta table
    df = pd.read_table("{}cell_metadata.csv".format(cur_dir), sep=',')

    # read the genes table 
    gene_df = pd.read_table("{}genes.csv".format(cur_dir), sep=',')
    gene_df.to_csv("{}genes.tsv".format(cur_dir), sep='\t', index=False, header=False)

    # just get the cell barcode and write to output file:
    df.cell_barcode.to_csv("{}barcodes.tsv".format(cur_dir), sep='\t', index=False, header=False)
    
    # transpose the expression matrix:
    X = io.mmread("{}DGE.mtx".format(cur_dir))
    X = X.transpose().tocsr()
    io.mmwrite("{}matrix.mtx".format(cur_dir), X)

/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch1/Sublibrary1_S1/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch1/Sublibrary2_S2/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch1/Sublibrary3_S3/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch1/Sublibrary4_S4/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch1/Sublibrary6_S6/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch1/Sublibrary7_S7/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch1/Sublibrary8_S8/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch1/Sublibrary5_S5/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch2/Sublibrary_3_S3/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/data/ADDS_2021/analysis/Batch2/Sublibrary_1_S1/all-well/DGE_unfiltered/
/dfs3b/swaruplab/smorabit/da

In [20]:
# write the sublibrary directories to an output file that we can index with our for loop in bash:
pd.Series(sublib_dirs).to_csv('sublibrary_dirs.csv', index=False, header=False)

In [21]:
len(sublib_dirs)

32

In [26]:
# run the cellbender script:
#! ./run_cellbender.sh

/bin/bash: ./run_cellbender.sh: No such file or directory


yee
